In [1]:
import pandas as pd
import numpy as np

from functools import reduce

from sqlalchemy import create_engine

from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

- Need to get users and their interactions so that I can
    - Group by the user ages/user tenure and interaction type

- Need to analyze how these different groups approach the different types of offers
    - Are they more likely to complete an offer

- Need to analyze which offers are more likely to be completed

# Loading data

In [2]:
conn = create_engine('sqlite:///data/raw_starbucks.db')

# offers.to_sql('offers', conn, if_exists='replace', index=False)
# users.to_sql('users', conn, if_exists='replace', index=False)
# interactions.to_sql('interactions', conn, if_exists='replace', index=False)

offers = pd.read_sql('SELECT * FROM offers', conn)

users = pd.read_sql('SELECT * FROM users', conn)
users['user_age_group'] = users['user_age_group'].fillna('Unknown')
users['user_age_group'] = users['user_age_group'].astype(str)
categories = ['Unknown'] + sorted(users['user_age_group'].unique().tolist())[:-1]
users['user_age_group'] = pd.Categorical(users['user_age_group'], categories=categories, ordered=True)

interactions = pd.read_sql('SELECT * FROM interactions', conn)

# User and interactions

1. Need to try and find out how many users have a transaction after receiving/viewing an offer
2. Need to try and find out how many users have a transaction without a prior offer

In [3]:
offer_received = interactions[ interactions['intxn_event_type'] == 'offer received' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_received_cols = [ "received" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_received.columns ]
offer_received.columns = offer_received_cols

offer_viewed = interactions[ interactions['intxn_event_type'] == 'offer viewed' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_viewed_cols = [ "viewed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_viewed.columns ]
offer_viewed.columns = offer_viewed_cols

offer_completed = interactions[ interactions['intxn_event_type'] == 'offer completed' ].copy().drop(['intxn_amount'], axis=1)
offer_completed_cols = [ "completed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_completed.columns ]
offer_completed.columns = offer_completed_cols

transactions = interactions[ interactions['intxn_event_type'] == 'transaction' ].copy().drop(['offer_id', 'intxn_reward'], axis=1)
transactions_cols = [ "transaction" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in transactions.columns ]
transactions.columns = transactions_cols


## Users with transactions but no offers received

In [11]:
transactions_no_promos = pd.merge( transactions, offer_received, on='user_id', how='left' )
transactions_no_promos = transactions_no_promos[ transactions_no_promos['offer_id'].isna() ]

transactions_no_promos

,user_id,transaction_event_type,transaction_time,transaction_amount,received_event_type,received_time,offer_id
4096,eb540099db834cf59001f83a4561aef3,transaction,6,4.74,NaN,NaN,NaN
14148,12ede229379747bd8d74ccdc20097ca3,transaction,18,16.62,NaN,NaN,NaN
124269,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,174,0.65,NaN,NaN,NaN
166666,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,222,1.91,NaN,NaN,NaN
238452,ae8111e7e8cd4b60a8d35c42c1110555,transaction,306,17.51,NaN,NaN,NaN
244267,eb540099db834cf59001f83a4561aef3,transaction,318,5.09,NaN,NaN,NaN
248661,3a4874d8f0ef42b9a1b72294902afea9,transaction,324,22.75,NaN,NaN,NaN
250562,12ede229379747bd8d74ccdc20097ca3,transaction,324,27.60,NaN,NaN,NaN
254212,ae8111e7e8cd4b60a8d35c42c1110555,transaction,330,28.70,NaN,NaN,NaN
294458,3a4874d8f0ef42b9a1b72294902afea9,transaction,378,27.20,NaN,NaN,NaN


## Users with offers received but no offers viewed

In [34]:
offer_not_viewed = pd.merge( offer_received, offer_viewed, on=['user_id', 'offer_id'], how='left' )
offer_not_viewed = offer_not_viewed[ offer_not_viewed['viewed_time'].isna() ]

display(offer_not_viewed)

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time
11,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN
26,65aba5c617294649aeb624da249e1ee5,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
29,868317b9be554cb18e50bc68484749a2,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
32,4beeb3ed64dd4898b0edf2f6b67426d3,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
34,25c906289d154b66bf579693f89481c9,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
...,...,...,...,...,...,...
95306,97ee6e7a12fe4064b260fa48bdd0330f,offer received,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
95307,bf3069b178fe40d789dd027901bc406b,offer received,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
95310,f653cf2d8bba42d0a53c2937ee2e5893,offer received,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
95317,cb23b66c56f64b109d673d5e56574529,offer received,576,2906b810c7d4411798c6938adc9daaa5,NaN,NaN


In [33]:
user_id = '868317b9be554cb18e50bc68484749a2'
offer_id = '2906b810c7d4411798c6938adc9daaa5'

display( interactions[ (interactions['user_id'] == user_id) & (interactions['offer_id']==offer_id) ].sort_values( by=['intxn_time'] ) )
display( transactions[ transactions['user_id'] == user_id ] )
display( offers[ offers['offer_id'] == offer_id ] )

,user_id,intxn_event_type,intxn_time,intxn_amount,offer_id,intxn_reward
20,868317b9be554cb18e50bc68484749a2,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
47585,868317b9be554cb18e50bc68484749a2,offer completed,132,NaN,2906b810c7d4411798c6938adc9daaa5,2.0


,user_id,transaction_event_type,transaction_time,transaction_amount
47584,868317b9be554cb18e50bc68484749a2,transaction,132,12.03
101461,868317b9be554cb18e50bc68484749a2,transaction,282,28.37
191218,868317b9be554cb18e50bc68484749a2,transaction,468,19.47
277107,868317b9be554cb18e50bc68484749a2,transaction,612,22.05


,offer_id,offer_notification_channel,offer_type,offer_spend_minimum,offer_reward,offer_duration
30,2906b810c7d4411798c6938adc9daaa5,web,discount,10,2,168
31,2906b810c7d4411798c6938adc9daaa5,email,discount,10,2,168
32,2906b810c7d4411798c6938adc9daaa5,mobile,discount,10,2,168


## Users with offers viewed but not completed

In [37]:
offer_not_completed = pd.merge( offer_viewed, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_not_completed = offer_not_completed[ offer_not_completed['completed_event_type'].isna() ]

display(offer_not_completed)

,user_id,viewed_event_type,viewed_time,offer_id,completed_event_type,completed_time,completed_reward
2,d1ede868e29245ea91818a903fec04c6,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
4,02c083884c7d45b39cc68e1314fec56c,offer viewed,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN
5,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
8,e528ceb341964128aaf58a59733ec2af,offer viewed,0,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN
9,262ad0fb526a4d53b572007da60cce24,offer viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN
...,...,...,...,...,...,...,...
66216,d56386cf344c4829bbf420d1895dca37,offer viewed,714,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
66217,9b51e8797290403b90d09d864dec4b94,offer viewed,714,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
66218,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,714,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
66219,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,714,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN


In [38]:
user_id = 'd1ede868e29245ea91818a903fec04c6'
offer_id = '5a8bc65990b245e5a138643cd4eb9837'

display( interactions[ (interactions['user_id'] == user_id) & (interactions['offer_id']==offer_id) ].sort_values( by=['intxn_time'] ) )
display( transactions[ transactions['user_id'] == user_id ] )
display( offers[ offers['offer_id'] == offer_id ] )

,user_id,intxn_event_type,intxn_time,intxn_amount,offer_id,intxn_reward
6490,d1ede868e29245ea91818a903fec04c6,offer received,0,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,0,NaN,5a8bc65990b245e5a138643cd4eb9837,NaN


,user_id,transaction_event_type,transaction_time,transaction_amount
22298,d1ede868e29245ea91818a903fec04c6,transaction,24,1.72
79425,d1ede868e29245ea91818a903fec04c6,transaction,198,4.12
98961,d1ede868e29245ea91818a903fec04c6,transaction,270,1.46
143577,d1ede868e29245ea91818a903fec04c6,transaction,384,8.64
145375,d1ede868e29245ea91818a903fec04c6,transaction,390,2.80


,offer_id,offer_notification_channel,offer_type,offer_spend_minimum,offer_reward,offer_duration
23,5a8bc65990b245e5a138643cd4eb9837,email,informational,0,0,72
24,5a8bc65990b245e5a138643cd4eb9837,mobile,informational,0,0,72
25,5a8bc65990b245e5a138643cd4eb9837,social,informational,0,0,72
